In [ ]:
import requests
from lxml import html

import redis
from mlrepricer import redisdb, helper
from random import randint
import time

from ipywidgets import Image

How you get started with redis.

In [ ]:
r = redis.StrictRedis(**helper.rediscred, decode_responses=True)
redisdb.server_start()

Iterate over pictures:

In [ ]:
m = []
for asin in r.scan_iter():
    if asin.startswith('pic'):
        m.append(list(r.smembers(asin))[0])

In [ ]:
# that would be one picture
pic = list(r.smembers('pic_B01F2RLCJ8'))[0]
#r.delete('pic_B01J670I36')
# lets use the list of pictures, you can change the index to play around
Image(value=m[23])

In [ ]:
def download_image(asin):
    country = 'de'
    site = requests.get(f'https://amazon.{country}/dp/{asin}')
    parser = html.fromstring(site.text)
    IMG = "//img[@id='landingImage']"
    image_link = parser.xpath(IMG)[0].get('src')
    image = requests.get(image_link).content
    assert isinstance(image, bytes)
    return image
    
def dump_into_redis(image):
    r.sadd(f'pic_{asin}', image)

In [ ]:
for asin in r.scan_iter():
    if asin != 'updated_asins' and not asin.startswith('pic'):
        if not r.exists('pic_'+asin):
            print(asin)
            time.sleep(randint(2,4))
            try:
                dump_into_redis(download_image(asin))
            except:
                time.sleep(randint(5, 6))